In [2]:
import numpy as np
import cv2
from urllib.request import Request, urlopen
import json, os

### Helper functions

In [89]:
def hex_to_bgr(h):
    h = h.lstrip('#')
    rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
    return (rgb[2], rgb[1], rgb[0])

In [90]:
def get_annotations(url):
    '''
    Get video annotations from scaleapi-results url
    '''
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read().decode('utf8').replace("'", '"')
    return json.loads(page)

In [91]:
def url_to_image(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read()
    image = np.asarray(bytearray(page), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

In [92]:
def draw(img, location, thickness, draw_transparent=False):
    
    if 'label' in location.keys() and location['label'] in object_lookup:
        color = object_lookup[location['label']]
    else:
        color = (0,255,0)
    
    if location['type'] == 'box':
        top = int(location['top'])
        left = int(location['left'])
        top_left = (left, top) # (384,0)
        bottom_right = (left + int(location['width']), top + int(location['height'])) # (510,128)
        cv2.rectangle(img, top_left, bottom_right, color, thickness)
    elif location['type'] == 'point':
        center_coordinates = (int(location['x']), int(location['y']))
        if draw_transparent:
            cv2.circle(img, center_coordinates, 3, color, 4)
            cv2.circle(img, center_coordinates, 15, color, 2)
        else:
            cv2.circle(img, center_coordinates, 20, color, 20)
            

In [93]:
thickness = 1

def main(url, annotations, out_file='out'):
    img = url_to_image(url)
    overlay = img.copy()
    
    # Apply annotations for boxes
    for annotation in annotations:
        draw(img, annotation, -1) # draw the opaque
    
    alpha = 0.8  # Transparency factor.

    # Following line overlays transparent rectangle over the image
    img_new = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)
    
    # Apply annotations for outlines
    for annotation in annotations:
        draw(img_new, annotation, thickness, True) 
    
    cv2.imwrite(out_file +'.png', img_new)

### Initialize 

In [96]:
# add all labels and color codes 
object_lookup = {
      "Car": hex_to_bgr('#FF9100'),
      "Bus": hex_to_bgr('#D501F9'),
      "Vehicle with Trailer": hex_to_bgr('#00E676'),
      "Truck": hex_to_bgr('#00E5FF'),
      "2-Wheeler": hex_to_bgr('#FFEA00'),
      "Building": hex_to_bgr('#6331FF'),
      "Other": hex_to_bgr('#FFEA00'),
      "Person": hex_to_bgr('#FFEA00'),
      "vehicle": hex_to_bgr('#FF9100')
}

In [127]:
# upload the task file prior
task_file_name = './parking_lot_boxes.json'

In [133]:
# get task json and annotations for entire video file
with open(task_file_name) as f:
    task_json = json.load(f)

In [129]:
annotations_url = task_json['attempts'][0]['reviews'][0]['reviewChanges']['annotations']['url']

In [130]:
annotations = get_annotations(annotations_url)

In [134]:
task_json['processed_attachments']


KeyError: 'processed_attachments'

In [ ]:
s3_conv_payload = {"inputObject":["https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83a00339db361_1610575269831_3840x2160_1.jpg"],"expireMs":36000000}

In [137]:
task_json['task']['processed_attachments']

['https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83a00339db361_1610575269831_3840x2160_1.jpg',
 'https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83a00339db361_1610575269831_3840x2160_2.jpg',
 'https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83a00339db361_1610575269831_3840x2160_3.jpg',
 'https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83a00339db361_1610575269831_3840x2160_4.jpg',
 'https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83a00339db361_1610575269831_3840x2160_5.jpg',
 'https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83a00339db361_1610575269831_3840x2160_6.jpg',
 'https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83a00339db361_1610575269831_3840x2160_7.jpg',
 'https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83a00339db361_1610575269831_3840x2160_8.jpg',
 'https://scale-us-attachments.s3.amazonaws.com/frame_task_5fff6d82c4c83

## Main

#### For raw links to s3 images using the corp s3 converstion tool

You need to grab all the processed_images links and convert them to downloadable links (via corp tool) saving them to the variable 'links'


In [108]:
for i, frame_url in enumerate(links):
    annotation = annotations[i]['annotations']
    padded_index_string = str(i).rjust(3,'0')
    output_filename = './out/img' + padded_index_string
    main(frame_url, annotation, output_filename)

#### specific for files that can be downloaded without s3 auth corp tool conversion


In [63]:
# specific for files that can be downloaded without s3 auth
for i, frame_url in enumerate(task_json['processed_attachments']):
    annotation = annotations[i]['annotations']
    padded_index_string = str(i).rjust(3,'0')
    output_filename = './out/img' + padded_index_string
    main(frame_url, annotation, output_filename)

HTTPError: HTTP Error 403: Forbidden

In [113]:
# processing video links

In [65]:
links = links.splitlines()

In [66]:
len(links)

243


# Conversion

In [109]:
frame_rate = task_json['params']['frame_rate']

In [110]:
print(frame_rate)

15


In [111]:
# set the frame rate manually
# frame_rate = 5

In [112]:
# convert all the images in the 'out' folder into a mp4 video
os.system("ffmpeg -r " + str(frame_rate) + " -i ./out/img%03d.png parking_lot.mp4")

0